In [5]:
import os

model_names = sorted(os.listdir('./Tables'))
model_names = [name.split('.xlsx')[0] for name in model_names]
model_names = [name.split('JETSON_')[1] for name in model_names]

In [4]:
model_sizes={
    'MultiModel-A-trt': 23 + 9 + 260,
    'MultiModel-B-trt': 23 + 1.3 + 1.3,
    'MultiModel-A': 23 + 9 + 385,
    'MultiModel-B': 23 + 1 + 1,
    'Mask2Former-Fp32': 190,
    'O2SFormer-Fp32': 125,
    'Resa-Fp16': 45,
    'Resa-Fp32': 90,
    'SegFormer-Fp16-trt': 21,
    'SegFormer-Fp16': 9,
    'SegFormer-Fp32-trt': 21,
    'SegFormer-Fp32': 16,
    'TwinLiteNet-Fp16-trt': 1.3,
    'TwinLiteNet-Fp16': 1,
    'TwinLiteNet-Fp32-trt': 2.6,
    'TwinLiteNet-Fp32': 2,
    'UFLDv2-Fp16-trt': 260,
    'UFLDv2-Fp32-trt': 530,
    'UFLDv2-Fp32': 385,
    'UperNet-Fp16': 120,
    'UperNet-Fp32': 240,
    'YoloV5s-Fp16-trt': 30,
    'YoloV5s-Fp16': 15,
    'YoloV5s-Fp32': 30,
    'YoloV7-Fp16': 75,
    'YoloV7-Fp32': 150,
    'YoloV8s-Fp16-trt': 64,
    'YoloV8s-Fp16': 23,
    'YoloV8s-Fp32-trt': 66,
    'YoloV8s-Fp32': 45,
    'Yolop-Fp16-trt': 40,
    'Yolop-Fp16': 16,
    'Yolop-Fp32-trt': 56,
    'Yolop-Fp32': 32,
    'YolopV2-Fp16-trt': 109,
    'YolopV2-Fp16': 78,
    'YolopV2-Fp32-trt': 247,
    'YolopV2-Fp32': 156 
}

In [7]:
import pandas as pd
models = {}
for model in model_names:
    models[model] = {}
    # models[model]['model'] = model
    table_o = pd.read_excel('./Tables/'+'JETSON_'+model+'.xlsx', sheet_name='Results', header = [0,1])
    mode_obj = 1 if 'Object detection' in table_o else 0
    mode_da = 1 if 'Drivable area' in table_o else 0
    mode_l = 1 if 'Lane marking' in table_o else 0
    if mode_obj:
        table_obj = table_o[['Unnamed: 0_level_0','Object detection']].dropna()
        models[model]['Object detection'] = {}
        models[model]['Object detection']['model'] = model
        models[model]['Object detection']['fps'] = table_obj[('Object detection', 'FPS')].iloc[0]
        models[model]['Object detection']['AP@50'] = table_obj[('Object detection', 'AP@50')].iloc[0]
        models[model]['Object detection']['AP@75'] = table_obj[('Object detection', 'AP@75')].iloc[0]
        models[model]['Object detection']['model_size'] = model_sizes[model]
    if mode_da:
        table_da = table_o[['Unnamed: 0_level_0','Drivable area']].dropna()
        table_da
        models[model]['Drivable area'] = {}
        models[model]['Drivable area']['model'] = model
        models[model]['Drivable area']['fps'] = table_da[('Drivable area', 'FPS')].iloc[0]
        models[model]['Drivable area']['iou'] = table_da[('Drivable area', 'IoU')].iloc[0]
        models[model]['Drivable area']['model_size'] = model_sizes[model]
    if mode_l:
        table_l = table_o[['Unnamed: 0_level_0','Lane marking']].dropna()
        models[model]['Lane marking'] = {}
        models[model]['Lane marking']['model'] = model
        models[model]['Lane marking']['fps'] = table_l[('Lane marking', 'FPS')].iloc[0]
        models[model]['Lane marking']['iou'] = table_l[('Lane marking', 'IoU')].iloc[0]
        models[model]['Lane marking']['model_size'] = model_sizes[model]
        
            

In [8]:
drivable_results = []
lane_results = []
obj_results = []

for model in models:
    if 'Drivable area' in models[model]:
        drivable_results.append(models[model]['Drivable area'])
    if 'Lane marking' in models[model]:
        lane_results.append(models[model]['Lane marking'])
    if 'Object detection' in models[model]:
        obj_results.append(models[model]['Object detection'])

In [9]:
columns_obj = pd.MultiIndex.from_tuples([('Object detection','AP@50'),('Object detection','AP@75'), ('Object detection','FPS'), ('Object detection', 'Model size')])

columns_drivable = pd.MultiIndex.from_tuples([('Drivable area','IoU'), ('Drivable area','FPS'), ('Drivable area', 'Model size')
                                                    ])

columns_lane = pd.MultiIndex.from_tuples([('Lane marking','IoU'), ('Lane marking','FPS'), ('Lane marking', 'Model size')
                                                    ])



df_obj = pd.DataFrame(columns=columns_obj)
df_drivable = pd.DataFrame(columns=columns_drivable)
df_lane = pd.DataFrame(columns=columns_lane)




In [10]:
for model_dict in drivable_results:
    df_drivable.loc[model_dict['model'], ('Drivable area','IoU')] = model_dict['iou']
    df_drivable.loc[model_dict['model'], ('Drivable area','FPS')] = model_dict['fps']
    df_drivable.loc[model_dict['model'], ('Drivable area','Model size')] = model_dict['model_size']
for model_dict in lane_results:
    df_lane.loc[model_dict['model'], ('Lane marking','IoU')] = model_dict['iou']
    df_lane.loc[model_dict['model'], ('Lane marking','FPS')] = model_dict['fps']
    df_lane.loc[model_dict['model'], ('Lane marking','Model size')] = model_dict['model_size']
for model_dict in obj_results:
    df_obj.loc[model_dict['model'], ('Object detection','AP@50')] = model_dict['AP@50']
    df_obj.loc[model_dict['model'], ('Object detection','AP@75')] = model_dict['AP@75']
    df_obj.loc[model_dict['model'], ('Object detection','FPS')] = model_dict['fps']
    df_obj.loc[model_dict['model'], ('Object detection','Model size')] = model_dict['model_size']

In [11]:
df_obj

Object detection                                
                            AP@50     AP@75        FPS Model size
MultiModel-A-trt         0.668324  0.528216  10.554784        292
MultiModel-A             0.668324  0.528216   9.199458        417
MultiModel-B-trt         0.668324  0.528216  22.512998       25.6
MultiModel-B             0.668324  0.528216   16.47326         25
YoloV8s-Fp16             0.668324  0.528216  49.496214         23
YolopV2-Fp16-trt         0.554573  0.419972  27.931497        109
YolopV2-Fp16             0.550567    0.4203  16.898148         78

In [12]:
with pd.ExcelWriter('First_run_detailed' + '.xlsx') as writer:
    df_obj.to_excel(writer, sheet_name='Object detection')
    df_drivable.to_excel(writer, sheet_name='Drivable area')
    df_lane.to_excel(writer, sheet_name='Lane marking')